In [ ]:
# Final code for reproducibility paper
# 1 is fraudulent
#%% Settings for runs
seeded_run = False
prototyping = False
num_epochs = 200

In [ ]:
#Importing all packages
import platform
import os

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_mean_pool






pc = platform.system()
if pc == "Darwin":
    os.chdir("/Users/lambertusvanzyl/Desktop/Reproducibility_paper")
else:
    os.chdir("/Users/Lambertus/Desktop/Reproducibility_paper")
    
if seeded_run:
    torch.manual_seed(42)
    np.random.seed(42)
else:
    seed = np.random.SeedSequence().entropy

# Importing custom libraries
from pre_processing import EllipticDataset, IBMAMLDataset_HiSmall, IBMAMLDataset_LiSmall, IBMAMLDataset_HiMedium, IBMAMLDataset_LiMedium, AMLSimDataset
from models import GCN, ModelWrapper

In [ ]:
if pc == "Darwin":
    #Processing elliptic dataset
    elliptic_data = EllipticDataset(root='/Users/lambertusvanzyl/Documents/Datasets/Elliptic_dataset')[0]
    #Processing IBM AML dataset
    IBM_data_HiSmall = IBMAMLDataset_HiSmall(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/HiSmall')[0]
    IBM_data_LiSmall = IBMAMLDataset_LiSmall(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/LiSmall')[0]
    IBM_data_HiMedium = IBMAMLDataset_HiMedium(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/HiMedium')[0]
    IBM_data_LiMedium = IBMAMLDataset_LiMedium(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/LiMedium')[0]
    #Processing AMLSim dataset
    AMLSim_data = AMLSimDataset(root='/Users/lambertusvanzyl/Documents/Datasets/AMLSim_dataset')[0]
else:
    #Processing elliptic dataset
    elliptic_data = EllipticDataset(root='/Users/Lambertus/Desktop/Datasets/Elliptic_dataset')[0]
    #Processing IBM AML dataset
    IBM_data_HiSmall = IBMAMLDataset_HiSmall(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/HiSmall')[0]
    IBM_data_LiSmall = IBMAMLDataset_LiSmall(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/LiSmall')[0]
    IBM_data_HiMedium = IBMAMLDataset_HiMedium(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/HiMedium')[0]
    IBM_data_LiMedium = IBMAMLDataset_LiMedium(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/LiMedium')[0]
    #Processing AMLSim dataset
    AMLSim_data = AMLSimDataset(root='/Users/Lambertus/Desktop/Datasets/AMLSim_dataset')[0]

c:\Users\Lambertus\Desktop\Reproducibility_paper\pre_processing.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_paths

In [ ]:
from torch.optim import Adam
if prototyping:
    data = elliptic_data
    #data = IBM_data
    # testing whether pre processing worked
    hidden_units = 64
    learning_rate=0.05
    loss = nn.CrossEntropyLoss()
    model = GCN(num_node_features=data.x.shape[1], num_classes=2, hidden_units=hidden_units)
    optimizer = Adam(model.parameters(), lr=learning_rate)
    model_wrapper = ModelWrapper(model, optimizer, loss)
    for i in range(1):
        train_loss = model_wrapper.train_step(data, data.train_perf_eval_mask)
        val_loss, val_metrics = model_wrapper.evaluate(data, data.val_perf_eval_mask)
        print(f"Epoch {i+1:03d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1 illicit: {val_metrics['f1_illicit']:.4f}")

In [ ]:
#from training_and_testing import train_and_test
#test_metrics, best_f1 = train_and_test(model_wrapper, data, data.test_perf_eval_mask, num_epochs=num_epochs)

In [ ]:
from hyperparameter_tuning import run_optimization
datasets = ["IBM_AML_HiSmall", "IBM_AML_LiSmall", "IBM_AML_HiMedium", "IBM_AML_LiMedium", "AMLSim"]
for x in datasets:
    match x:
        case "Elliptic":
            data_for_optimization = "Elliptic"
            data = elliptic_data
        case "IBM_AML_HiSmall":
            data_for_optimization = "IBM_AML_HiSmall"
            data = IBM_data_HiSmall
        case "IBM_AML_LiSmall":
            data_for_optimization = "IBM_AML_LiSmall"
            data = IBM_data_LiSmall
        case "IBM_AML_HiMedium":
            data_for_optimization = "IBM_AML_HiMedium"
            data = IBM_data_HiMedium
        case "IBM_AML_LiMedium":
            data_for_optimization = "IBM_AML_LiMedium"
            data = IBM_data_LiMedium
        case "AMLSim":
            data_for_optimization = "AMLSim"
            data = AMLSim_data
            
    def save_testing_results_csv(results, path=f"{data_for_optimization}_testing_results.csv"):
        df = pd.DataFrame(results)
        df.to_csv(f"csv_results/{data_for_optimization}_testing_results.csv", index=False)
    model_parameters, testing_results = run_optimization(
        models=['MLP', 'GCN', 'GAT', 'GIN'],
        data=data,
        train_perf_eval=data.train_perf_eval_mask,
        val_perf_eval=data.val_perf_eval_mask,
        test_perf_eval=data.test_perf_eval_mask,
        train_mask=data.train_mask,
        val_mask=data.val_mask,
        data_for_optimization=data_for_optimization
    )
    save_testing_results_csv(testing_results, path=f"{data_for_optimization}_testing_results.csv")

Models:   0%|          | 0/4 [00:00<?, ?model/s][I 2025-11-09 16:51:18,622] A new study created in RDB with name: MLP_optimization on IBM_AML_HiSmall dataset


Study 'MLP_optimization on IBM_AML_HiSmall dataset' found with only 5 trials (< 50). Deleting study.


[I 2025-11-09 16:51:38,594] Trial 0 finished with value: 0.002212773535762784 and parameters: {'learning_rate': 0.005664877475148562, 'weight_decay': 0.0002346433083775456, 'loss_type': 'focal', 'gamma_focal': 3.9819800887779984, 'early_stop_patience': 10, 'early_stop_min_delta': 0.0014463461549868047, 'num_epochs': 189, 'hidden_units': 119}. Best is trial 0 with value: 0.002212773535762784.
c:\Users\Lambertus\anaconda3\envs\Cuda\lib\site-packages\optuna\trial\_trial.py:678: RuntimeWarning: Inconsistent parameter values for distribution with name "learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'step': None, 'low': 0.005, 'high': 0.01, 'log': False}
  warnings.warn(
[W 2025-11-09 16:51:39,291] Trial 1 failed with parameters: {'learning_rate': 0.00930347396

ValueError: Loss is NaN, stopping training

In [ ]:
# Final code for reproducibility paper
# 1 is fraudulent
#%% Settings for runs
seeded_run = False
prototyping = False
num_epochs = 200

In [ ]:
#Importing all packages
import platform
import os

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_mean_pool






pc = platform.system()
if pc == "Darwin":
    os.chdir("/Users/lambertusvanzyl/Desktop/Reproducibility_paper")
else:
    os.chdir("/Users/Lambertus/Desktop/Reproducibility_paper")
    
if seeded_run:
    torch.manual_seed(42)
    np.random.seed(42)
else:
    seed = np.random.SeedSequence().entropy

# Importing custom libraries
from pre_processing import EllipticDataset, IBMAMLDataset_HiSmall, IBMAMLDataset_LiSmall, IBMAMLDataset_HiMedium, IBMAMLDataset_LiMedium, AMLSimDataset
from models import GCN, ModelWrapper

In [ ]:
if pc == "Darwin":
    #Processing elliptic dataset
    elliptic_data = EllipticDataset(root='/Users/lambertusvanzyl/Documents/Datasets/Elliptic_dataset')[0]
    #Processing IBM AML dataset
    IBM_data_HiSmall = IBMAMLDataset_HiSmall(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/HiSmall')[0]
    IBM_data_LiSmall = IBMAMLDataset_LiSmall(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/LiSmall')[0]
    IBM_data_HiMedium = IBMAMLDataset_HiMedium(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/HiMedium')[0]
    IBM_data_LiMedium = IBMAMLDataset_LiMedium(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/LiMedium')[0]
    #Processing AMLSim dataset
    AMLSim_data = AMLSimDataset(root='/Users/lambertusvanzyl/Documents/Datasets/AMLSim_dataset')[0]
else:
    #Processing elliptic dataset
    elliptic_data = EllipticDataset(root='/Users/Lambertus/Desktop/Datasets/Elliptic_dataset')[0]
    #Processing IBM AML dataset
    IBM_data_HiSmall = IBMAMLDataset_HiSmall(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/HiSmall')[0]
    IBM_data_LiSmall = IBMAMLDataset_LiSmall(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/LiSmall')[0]
    IBM_data_HiMedium = IBMAMLDataset_HiMedium(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/HiMedium')[0]
    IBM_data_LiMedium = IBMAMLDataset_LiMedium(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/LiMedium')[0]
    #Processing AMLSim dataset
    AMLSim_data = AMLSimDataset(root='/Users/Lambertus/Desktop/Datasets/AMLSim_dataset')[0]

c:\Users\Lambertus\Desktop\Reproducibility_paper\pre_processing.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_paths

In [ ]:
from torch.optim import Adam
if prototyping:
    data = elliptic_data
    #data = IBM_data
    # testing whether pre processing worked
    hidden_units = 64
    learning_rate=0.05
    loss = nn.CrossEntropyLoss()
    model = GCN(num_node_features=data.x.shape[1], num_classes=2, hidden_units=hidden_units)
    optimizer = Adam(model.parameters(), lr=learning_rate)
    model_wrapper = ModelWrapper(model, optimizer, loss)
    for i in range(1):
        train_loss = model_wrapper.train_step(data, data.train_perf_eval_mask)
        val_loss, val_metrics = model_wrapper.evaluate(data, data.val_perf_eval_mask)
        print(f"Epoch {i+1:03d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1 illicit: {val_metrics['f1_illicit']:.4f}")

In [ ]:
#from training_and_testing import train_and_test
#test_metrics, best_f1 = train_and_test(model_wrapper, data, data.test_perf_eval_mask, num_epochs=num_epochs)

In [ ]:
from hyperparameter_tuning import run_optimization
datasets = ["IBM_AML_HiSmall", "IBM_AML_LiSmall", "IBM_AML_HiMedium", "IBM_AML_LiMedium", "AMLSim"]
for x in datasets:
    match x:
        case "Elliptic":
            data_for_optimization = "Elliptic"
            data = elliptic_data
        case "IBM_AML_HiSmall":
            data_for_optimization = "IBM_AML_HiSmall"
            data = IBM_data_HiSmall
        case "IBM_AML_LiSmall":
            data_for_optimization = "IBM_AML_LiSmall"
            data = IBM_data_LiSmall
        case "IBM_AML_HiMedium":
            data_for_optimization = "IBM_AML_HiMedium"
            data = IBM_data_HiMedium
        case "IBM_AML_LiMedium":
            data_for_optimization = "IBM_AML_LiMedium"
            data = IBM_data_LiMedium
        case "AMLSim":
            data_for_optimization = "AMLSim"
            data = AMLSim_data
            
    def save_testing_results_csv(results, path=f"{data_for_optimization}_testing_results.csv"):
        df = pd.DataFrame(results)
        df.to_csv(f"csv_results/{data_for_optimization}_testing_results.csv", index=False)
    model_parameters, testing_results = run_optimization(
        models=['MLP', 'GCN', 'GAT', 'GIN'],
        data=data,
        train_perf_eval=data.train_perf_eval_mask,
        val_perf_eval=data.val_perf_eval_mask,
        test_perf_eval=data.test_perf_eval_mask,
        train_mask=data.train_mask,
        val_mask=data.val_mask,
        data_for_optimization=data_for_optimization
    )
    save_testing_results_csv(testing_results, path=f"{data_for_optimization}_testing_results.csv")

Models:   0%|          | 0/4 [00:00<?, ?model/s][I 2025-11-09 16:55:34,325] A new study created in RDB with name: MLP_optimization on IBM_AML_HiSmall dataset


Study 'MLP_optimization on IBM_AML_HiSmall dataset' found with only 2 trials (< 50). Deleting study.


[I 2025-11-09 16:56:56,368] Trial 0 finished with value: 0.003975453154332771 and parameters: {'learning_rate': 0.021669494871534523, 'weight_decay': 0.0002902301541400427, 'loss_type': 'focal', 'gamma_focal': 4.63079860961585, 'early_stop_patience': 19, 'early_stop_min_delta': 0.0001915910730366131, 'num_epochs': 69, 'hidden_units': 59}. Best is trial 0 with value: 0.003975453154332771.
[I 2025-11-09 16:57:28,733] Trial 1 finished with value: 0.002212773535762784 and parameters: {'learning_rate': 0.02938194059203128, 'weight_decay': 0.0004438310367819097, 'loss_type': 'focal', 'gamma_focal': 3.2966824199433593, 'early_stop_patience': 19, 'early_stop_min_delta': 0.00010239782186419199, 'num_epochs': 102, 'hidden_units': 99}. Best is trial 0 with value: 0.003975453154332771.
[I 2025-11-09 16:57:43,874] Trial 2 finished with value: 0.0 and parameters: {'learning_rate': 0.012757410121768904, 'weight_decay': 0.0009445390308912859, 'loss_type': 'focal', 'gamma_focal': 2.605997774304115, 'ea

ValueError: Loss is NaN, stopping training

In [ ]:
# Final code for reproducibility paper
# 1 is fraudulent
#%% Settings for runs
seeded_run = False
prototyping = False
num_epochs = 200

In [ ]:
#Importing all packages
import platform
import os

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_mean_pool






pc = platform.system()
if pc == "Darwin":
    os.chdir("/Users/lambertusvanzyl/Desktop/Reproducibility_paper")
else:
    os.chdir("/Users/Lambertus/Desktop/Reproducibility_paper")
    
if seeded_run:
    torch.manual_seed(42)
    np.random.seed(42)
else:
    seed = np.random.SeedSequence().entropy

# Importing custom libraries
from pre_processing import EllipticDataset, IBMAMLDataset_HiSmall, IBMAMLDataset_LiSmall, IBMAMLDataset_HiMedium, IBMAMLDataset_LiMedium, AMLSimDataset
from models import GCN, ModelWrapper

In [ ]:
if pc == "Darwin":
    #Processing elliptic dataset
    elliptic_data = EllipticDataset(root='/Users/lambertusvanzyl/Documents/Datasets/Elliptic_dataset')[0]
    #Processing IBM AML dataset
    IBM_data_HiSmall = IBMAMLDataset_HiSmall(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/HiSmall')[0]
    IBM_data_LiSmall = IBMAMLDataset_LiSmall(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/LiSmall')[0]
    IBM_data_HiMedium = IBMAMLDataset_HiMedium(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/HiMedium')[0]
    IBM_data_LiMedium = IBMAMLDataset_LiMedium(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/LiMedium')[0]
    #Processing AMLSim dataset
    AMLSim_data = AMLSimDataset(root='/Users/lambertusvanzyl/Documents/Datasets/AMLSim_dataset')[0]
else:
    #Processing elliptic dataset
    elliptic_data = EllipticDataset(root='/Users/Lambertus/Desktop/Datasets/Elliptic_dataset')[0]
    #Processing IBM AML dataset
    IBM_data_HiSmall = IBMAMLDataset_HiSmall(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/HiSmall')[0]
    IBM_data_LiSmall = IBMAMLDataset_LiSmall(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/LiSmall')[0]
    IBM_data_HiMedium = IBMAMLDataset_HiMedium(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/HiMedium')[0]
    IBM_data_LiMedium = IBMAMLDataset_LiMedium(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/LiMedium')[0]
    #Processing AMLSim dataset
    AMLSim_data = AMLSimDataset(root='/Users/Lambertus/Desktop/Datasets/AMLSim_dataset')[0]

c:\Users\Lambertus\Desktop\Reproducibility_paper\pre_processing.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_paths

In [ ]:
from torch.optim import Adam
if prototyping:
    data = elliptic_data
    #data = IBM_data
    # testing whether pre processing worked
    hidden_units = 64
    learning_rate=0.05
    loss = nn.CrossEntropyLoss()
    model = GCN(num_node_features=data.x.shape[1], num_classes=2, hidden_units=hidden_units)
    optimizer = Adam(model.parameters(), lr=learning_rate)
    model_wrapper = ModelWrapper(model, optimizer, loss)
    for i in range(1):
        train_loss = model_wrapper.train_step(data, data.train_perf_eval_mask)
        val_loss, val_metrics = model_wrapper.evaluate(data, data.val_perf_eval_mask)
        print(f"Epoch {i+1:03d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1 illicit: {val_metrics['f1_illicit']:.4f}")

In [ ]:
#from training_and_testing import train_and_test
#test_metrics, best_f1 = train_and_test(model_wrapper, data, data.test_perf_eval_mask, num_epochs=num_epochs)

In [ ]:
from hyperparameter_tuning import run_optimization
datasets = ["IBM_AML_HiSmall", "IBM_AML_LiSmall", "IBM_AML_HiMedium", "IBM_AML_LiMedium", "AMLSim"]
for x in datasets:
    match x:
        case "Elliptic":
            data_for_optimization = "Elliptic"
            data = elliptic_data
        case "IBM_AML_HiSmall":
            data_for_optimization = "IBM_AML_HiSmall"
            data = IBM_data_HiSmall
        case "IBM_AML_LiSmall":
            data_for_optimization = "IBM_AML_LiSmall"
            data = IBM_data_LiSmall
        case "IBM_AML_HiMedium":
            data_for_optimization = "IBM_AML_HiMedium"
            data = IBM_data_HiMedium
        case "IBM_AML_LiMedium":
            data_for_optimization = "IBM_AML_LiMedium"
            data = IBM_data_LiMedium
        case "AMLSim":
            data_for_optimization = "AMLSim"
            data = AMLSim_data
            
    def save_testing_results_csv(results, path=f"{data_for_optimization}_testing_results.csv"):
        df = pd.DataFrame(results)
        df.to_csv(f"csv_results/{data_for_optimization}_testing_results.csv", index=False)
    model_parameters, testing_results = run_optimization(
        models=['MLP', 'GCN', 'GAT', 'GIN'],
        data=data,
        train_perf_eval=data.train_perf_eval_mask,
        val_perf_eval=data.val_perf_eval_mask,
        test_perf_eval=data.test_perf_eval_mask,
        train_mask=data.train_mask,
        val_mask=data.val_mask,
        data_for_optimization=data_for_optimization
    )
    save_testing_results_csv(testing_results, path=f"{data_for_optimization}_testing_results.csv")

Models:   0%|          | 0/4 [00:00<?, ?model/s]

Study 'MLP_optimization on IBM_AML_HiSmall dataset' found with only 4 trials (< 50). Deleting study.


[I 2025-11-09 16:58:08,959] A new study created in RDB with name: MLP_optimization on IBM_AML_HiSmall dataset
c:\Users\Lambertus\anaconda3\envs\Cuda\lib\site-packages\optuna\trial\_trial.py:678: RuntimeWarning: Inconsistent parameter values for distribution with name "learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'step': None, 'low': 0.005, 'high': 0.05, 'log': False}
  warnings.warn(
[W 2025-11-09 16:58:09,801] Trial 0 failed with parameters: {'learning_rate': 0.040604132084094566, 'weight_decay': 0.0004576700325318266, 'loss_type': 'cross_entropy', 'early_stop_patience': 28, 'early_stop_min_delta': 0.0005056325299229556, 'num_epochs': 107, 'hidden_units': 37} because of the following error: ValueError('Loss is NaN, stopping training').
Traceback (most 

ValueError: Loss is NaN, stopping training

In [ ]:
# Final code for reproducibility paper
# 1 is fraudulent
#%% Settings for runs
seeded_run = False
prototyping = False
num_epochs = 200

In [ ]:
#Importing all packages
import platform
import os

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_mean_pool






pc = platform.system()
if pc == "Darwin":
    os.chdir("/Users/lambertusvanzyl/Desktop/Reproducibility_paper")
else:
    os.chdir("/Users/Lambertus/Desktop/Reproducibility_paper")
    
if seeded_run:
    torch.manual_seed(42)
    np.random.seed(42)
else:
    seed = np.random.SeedSequence().entropy

# Importing custom libraries
from pre_processing import EllipticDataset, IBMAMLDataset_HiSmall, IBMAMLDataset_LiSmall, IBMAMLDataset_HiMedium, IBMAMLDataset_LiMedium, AMLSimDataset
from models import GCN, ModelWrapper

In [ ]:
if pc == "Darwin":
    #Processing elliptic dataset
    elliptic_data = EllipticDataset(root='/Users/lambertusvanzyl/Documents/Datasets/Elliptic_dataset')[0]
    #Processing IBM AML dataset
    IBM_data_HiSmall = IBMAMLDataset_HiSmall(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/HiSmall')[0]
    IBM_data_LiSmall = IBMAMLDataset_LiSmall(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/LiSmall')[0]
    IBM_data_HiMedium = IBMAMLDataset_HiMedium(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/HiMedium')[0]
    IBM_data_LiMedium = IBMAMLDataset_LiMedium(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/LiMedium')[0]
    #Processing AMLSim dataset
    AMLSim_data = AMLSimDataset(root='/Users/lambertusvanzyl/Documents/Datasets/AMLSim_dataset')[0]
else:
    #Processing elliptic dataset
    elliptic_data = EllipticDataset(root='/Users/Lambertus/Desktop/Datasets/Elliptic_dataset')[0]
    #Processing IBM AML dataset
    IBM_data_HiSmall = IBMAMLDataset_HiSmall(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/HiSmall')[0]
    IBM_data_LiSmall = IBMAMLDataset_LiSmall(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/LiSmall')[0]
    IBM_data_HiMedium = IBMAMLDataset_HiMedium(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/HiMedium')[0]
    IBM_data_LiMedium = IBMAMLDataset_LiMedium(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/LiMedium')[0]
    #Processing AMLSim dataset
    AMLSim_data = AMLSimDataset(root='/Users/Lambertus/Desktop/Datasets/AMLSim_dataset')[0]

c:\Users\Lambertus\Desktop\Reproducibility_paper\pre_processing.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_paths

In [ ]:
from torch.optim import Adam
if prototyping:
    data = elliptic_data
    #data = IBM_data
    # testing whether pre processing worked
    hidden_units = 64
    learning_rate=0.05
    loss = nn.CrossEntropyLoss()
    model = GCN(num_node_features=data.x.shape[1], num_classes=2, hidden_units=hidden_units)
    optimizer = Adam(model.parameters(), lr=learning_rate)
    model_wrapper = ModelWrapper(model, optimizer, loss)
    for i in range(1):
        train_loss = model_wrapper.train_step(data, data.train_perf_eval_mask)
        val_loss, val_metrics = model_wrapper.evaluate(data, data.val_perf_eval_mask)
        print(f"Epoch {i+1:03d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1 illicit: {val_metrics['f1_illicit']:.4f}")

In [ ]:
#from training_and_testing import train_and_test
#test_metrics, best_f1 = train_and_test(model_wrapper, data, data.test_perf_eval_mask, num_epochs=num_epochs)

In [ ]:
from hyperparameter_tuning import run_optimization
datasets = ["IBM_AML_HiSmall", "IBM_AML_LiSmall", "IBM_AML_HiMedium", "IBM_AML_LiMedium", "AMLSim"]
for x in datasets:
    match x:
        case "Elliptic":
            data_for_optimization = "Elliptic"
            data = elliptic_data
        case "IBM_AML_HiSmall":
            data_for_optimization = "IBM_AML_HiSmall"
            data = IBM_data_HiSmall
        case "IBM_AML_LiSmall":
            data_for_optimization = "IBM_AML_LiSmall"
            data = IBM_data_LiSmall
        case "IBM_AML_HiMedium":
            data_for_optimization = "IBM_AML_HiMedium"
            data = IBM_data_HiMedium
        case "IBM_AML_LiMedium":
            data_for_optimization = "IBM_AML_LiMedium"
            data = IBM_data_LiMedium
        case "AMLSim":
            data_for_optimization = "AMLSim"
            data = AMLSim_data
            
    def save_testing_results_csv(results, path=f"{data_for_optimization}_testing_results.csv"):
        df = pd.DataFrame(results)
        df.to_csv(f"csv_results/{data_for_optimization}_testing_results.csv", index=False)
    model_parameters, testing_results = run_optimization(
        models=['MLP', 'GCN', 'GAT', 'GIN'],
        data=data,
        train_perf_eval=data.train_perf_eval_mask,
        val_perf_eval=data.val_perf_eval_mask,
        test_perf_eval=data.test_perf_eval_mask,
        train_mask=data.train_mask,
        val_mask=data.val_mask,
        data_for_optimization=data_for_optimization
    )
    save_testing_results_csv(testing_results, path=f"{data_for_optimization}_testing_results.csv")

Models:   0%|          | 0/4 [00:00<?, ?model/s][I 2025-11-09 16:58:53,911] A new study created in RDB with name: MLP_optimization on IBM_AML_HiSmall dataset


Study 'MLP_optimization on IBM_AML_HiSmall dataset' found with only 1 trials (< 50). Deleting study.


[I 2025-11-09 16:59:11,147] Trial 0 finished with value: 0.0 and parameters: {'learning_rate': 0.009127917079221483, 'weight_decay': 0.000418000615334913, 'loss_type': 'focal', 'gamma_focal': 0.4516223509587137, 'early_stop_patience': 7, 'early_stop_min_delta': 0.0026769112693839753, 'num_epochs': 168, 'hidden_units': 191}. Best is trial 0 with value: 0.0.
[I 2025-11-09 17:00:18,898] Trial 1 finished with value: 0.00228755233067736 and parameters: {'learning_rate': 0.00605702922466748, 'weight_decay': 0.0001431935664773733, 'loss_type': 'focal', 'gamma_focal': 3.909549867231278, 'early_stop_patience': 37, 'early_stop_min_delta': 0.0006237773472984972, 'num_epochs': 145, 'hidden_units': 69}. Best is trial 1 with value: 0.00228755233067736.
[I 2025-11-09 17:01:25,992] Trial 2 finished with value: 0.0023065143126398866 and parameters: {'learning_rate': 0.006902215439367676, 'weight_decay': 0.0008104505319847402, 'loss_type': 'focal', 'gamma_focal': 4.419245156905671, 'early_stop_patience'

ValueError: Loss is NaN, stopping training

In [ ]:
# Final code for reproducibility paper
# 1 is fraudulent
#%% Settings for runs
seeded_run = False
prototyping = False
num_epochs = 200

In [ ]:
#Importing all packages
import platform
import os

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_mean_pool






pc = platform.system()
if pc == "Darwin":
    os.chdir("/Users/lambertusvanzyl/Desktop/Reproducibility_paper")
else:
    os.chdir("/Users/Lambertus/Desktop/Reproducibility_paper")
    
if seeded_run:
    torch.manual_seed(42)
    np.random.seed(42)
else:
    seed = np.random.SeedSequence().entropy

# Importing custom libraries
from pre_processing import EllipticDataset, IBMAMLDataset_HiSmall, IBMAMLDataset_LiSmall, IBMAMLDataset_HiMedium, IBMAMLDataset_LiMedium, AMLSimDataset
from models import GCN, ModelWrapper

In [ ]:
if pc == "Darwin":
    #Processing elliptic dataset
    elliptic_data = EllipticDataset(root='/Users/lambertusvanzyl/Documents/Datasets/Elliptic_dataset')[0]
    #Processing IBM AML dataset
    IBM_data_HiSmall = IBMAMLDataset_HiSmall(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/HiSmall')[0]
    IBM_data_LiSmall = IBMAMLDataset_LiSmall(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/LiSmall')[0]
    IBM_data_HiMedium = IBMAMLDataset_HiMedium(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/HiMedium')[0]
    IBM_data_LiMedium = IBMAMLDataset_LiMedium(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/LiMedium')[0]
    #Processing AMLSim dataset
    AMLSim_data = AMLSimDataset(root='/Users/lambertusvanzyl/Documents/Datasets/AMLSim_dataset')[0]
else:
    #Processing elliptic dataset
    elliptic_data = EllipticDataset(root='/Users/Lambertus/Desktop/Datasets/Elliptic_dataset')[0]
    #Processing IBM AML dataset
    IBM_data_HiSmall = IBMAMLDataset_HiSmall(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/HiSmall')[0]
    IBM_data_LiSmall = IBMAMLDataset_LiSmall(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/LiSmall')[0]
    IBM_data_HiMedium = IBMAMLDataset_HiMedium(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/HiMedium')[0]
    IBM_data_LiMedium = IBMAMLDataset_LiMedium(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/LiMedium')[0]
    #Processing AMLSim dataset
    AMLSim_data = AMLSimDataset(root='/Users/Lambertus/Desktop/Datasets/AMLSim_dataset')[0]

c:\Users\Lambertus\Desktop\Reproducibility_paper\pre_processing.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_paths

In [ ]:
from torch.optim import Adam
if prototyping:
    data = elliptic_data
    #data = IBM_data
    # testing whether pre processing worked
    hidden_units = 64
    learning_rate=0.05
    loss = nn.CrossEntropyLoss()
    model = GCN(num_node_features=data.x.shape[1], num_classes=2, hidden_units=hidden_units)
    optimizer = Adam(model.parameters(), lr=learning_rate)
    model_wrapper = ModelWrapper(model, optimizer, loss)
    for i in range(1):
        train_loss = model_wrapper.train_step(data, data.train_perf_eval_mask)
        val_loss, val_metrics = model_wrapper.evaluate(data, data.val_perf_eval_mask)
        print(f"Epoch {i+1:03d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1 illicit: {val_metrics['f1_illicit']:.4f}")

In [ ]:
#from training_and_testing import train_and_test
#test_metrics, best_f1 = train_and_test(model_wrapper, data, data.test_perf_eval_mask, num_epochs=num_epochs)

In [ ]:
from hyperparameter_tuning import run_optimization
datasets = ["IBM_AML_HiSmall", "IBM_AML_LiSmall", "IBM_AML_HiMedium", "IBM_AML_LiMedium", "AMLSim"]
for x in datasets:
    match x:
        case "Elliptic":
            data_for_optimization = "Elliptic"
            data = elliptic_data
        case "IBM_AML_HiSmall":
            data_for_optimization = "IBM_AML_HiSmall"
            data = IBM_data_HiSmall
        case "IBM_AML_LiSmall":
            data_for_optimization = "IBM_AML_LiSmall"
            data = IBM_data_LiSmall
        case "IBM_AML_HiMedium":
            data_for_optimization = "IBM_AML_HiMedium"
            data = IBM_data_HiMedium
        case "IBM_AML_LiMedium":
            data_for_optimization = "IBM_AML_LiMedium"
            data = IBM_data_LiMedium
        case "AMLSim":
            data_for_optimization = "AMLSim"
            data = AMLSim_data
            
    def save_testing_results_csv(results, path=f"{data_for_optimization}_testing_results.csv"):
        df = pd.DataFrame(results)
        df.to_csv(f"csv_results/{data_for_optimization}_testing_results.csv", index=False)
    model_parameters, testing_results = run_optimization(
        models=['MLP', 'GCN', 'GAT', 'GIN'],
        data=data,
        train_perf_eval=data.train_perf_eval_mask,
        val_perf_eval=data.val_perf_eval_mask,
        test_perf_eval=data.test_perf_eval_mask,
        train_mask=data.train_mask,
        val_mask=data.val_mask,
        data_for_optimization=data_for_optimization
    )
    save_testing_results_csv(testing_results, path=f"{data_for_optimization}_testing_results.csv")

Models:   0%|          | 0/4 [00:00<?, ?model/s][I 2025-11-09 17:01:55,886] A new study created in RDB with name: MLP_optimization on IBM_AML_HiSmall dataset


Study 'MLP_optimization on IBM_AML_HiSmall dataset' found with only 4 trials (< 50). Deleting study.


[I 2025-11-09 17:02:50,685] Trial 0 finished with value: 0.0031369426478335704 and parameters: {'learning_rate': 0.007845666728124201, 'weight_decay': 0.0006419031879792016, 'loss_type': 'focal', 'gamma_focal': 0.8532379192751582, 'early_stop_patience': 14, 'early_stop_min_delta': 0.00010429438731937949, 'num_epochs': 130, 'hidden_units': 169}. Best is trial 0 with value: 0.0031369426478335704.
[I 2025-11-09 17:03:03,946] Trial 1 finished with value: 0.0023411790703369026 and parameters: {'learning_rate': 0.00612679777268512, 'weight_decay': 0.00047650927536497937, 'loss_type': 'focal', 'gamma_focal': 4.3589598978296005, 'early_stop_patience': 5, 'early_stop_min_delta': 0.0014922954384255472, 'num_epochs': 208, 'hidden_units': 73}. Best is trial 0 with value: 0.0031369426478335704.
[I 2025-11-09 17:03:39,448] Trial 2 finished with value: 0.002212773535762784 and parameters: {'learning_rate': 0.00770889938067742, 'weight_decay': 0.0004399263022253571, 'loss_type': 'focal', 'gamma_focal'

ValueError: Loss is NaN, stopping training

In [ ]:
# Final code for reproducibility paper
# 1 is fraudulent
#%% Settings for runs
seeded_run = False
prototyping = False
num_epochs = 200

In [ ]:
#Importing all packages
import platform
import os

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_mean_pool






pc = platform.system()
if pc == "Darwin":
    os.chdir("/Users/lambertusvanzyl/Desktop/Reproducibility_paper")
else:
    os.chdir("/Users/Lambertus/Desktop/Reproducibility_paper")
    
if seeded_run:
    torch.manual_seed(42)
    np.random.seed(42)
else:
    seed = np.random.SeedSequence().entropy

# Importing custom libraries
from pre_processing import EllipticDataset, IBMAMLDataset_HiSmall, IBMAMLDataset_LiSmall, IBMAMLDataset_HiMedium, IBMAMLDataset_LiMedium, AMLSimDataset
from models import GCN, ModelWrapper

In [ ]:
if pc == "Darwin":
    #Processing elliptic dataset
    elliptic_data = EllipticDataset(root='/Users/lambertusvanzyl/Documents/Datasets/Elliptic_dataset')[0]
    #Processing IBM AML dataset
    IBM_data_HiSmall = IBMAMLDataset_HiSmall(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/HiSmall')[0]
    IBM_data_LiSmall = IBMAMLDataset_LiSmall(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/LiSmall')[0]
    IBM_data_HiMedium = IBMAMLDataset_HiMedium(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/HiMedium')[0]
    IBM_data_LiMedium = IBMAMLDataset_LiMedium(root='/Users/lambertusvanzyl/Documents/Datasets/IBM_AML_dataset/LiMedium')[0]
    #Processing AMLSim dataset
    AMLSim_data = AMLSimDataset(root='/Users/lambertusvanzyl/Documents/Datasets/AMLSim_dataset')[0]
else:
    #Processing elliptic dataset
    elliptic_data = EllipticDataset(root='/Users/Lambertus/Desktop/Datasets/Elliptic_dataset')[0]
    #Processing IBM AML dataset
    IBM_data_HiSmall = IBMAMLDataset_HiSmall(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/HiSmall')[0]
    IBM_data_LiSmall = IBMAMLDataset_LiSmall(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/LiSmall')[0]
    IBM_data_HiMedium = IBMAMLDataset_HiMedium(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/HiMedium')[0]
    IBM_data_LiMedium = IBMAMLDataset_LiMedium(root='/Users/Lambertus/Desktop/Datasets/IBM_AML_dataset/LiMedium')[0]
    #Processing AMLSim dataset
    AMLSim_data = AMLSimDataset(root='/Users/Lambertus/Desktop/Datasets/AMLSim_dataset')[0]

c:\Users\Lambertus\Desktop\Reproducibility_paper\pre_processing.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_paths

In [ ]:
from torch.optim import Adam
if prototyping:
    data = elliptic_data
    #data = IBM_data
    # testing whether pre processing worked
    hidden_units = 64
    learning_rate=0.05
    loss = nn.CrossEntropyLoss()
    model = GCN(num_node_features=data.x.shape[1], num_classes=2, hidden_units=hidden_units)
    optimizer = Adam(model.parameters(), lr=learning_rate)
    model_wrapper = ModelWrapper(model, optimizer, loss)
    for i in range(1):
        train_loss = model_wrapper.train_step(data, data.train_perf_eval_mask)
        val_loss, val_metrics = model_wrapper.evaluate(data, data.val_perf_eval_mask)
        print(f"Epoch {i+1:03d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1 illicit: {val_metrics['f1_illicit']:.4f}")

In [ ]:
#from training_and_testing import train_and_test
#test_metrics, best_f1 = train_and_test(model_wrapper, data, data.test_perf_eval_mask, num_epochs=num_epochs)

In [ ]:
from hyperparameter_tuning import run_optimization
datasets = ["IBM_AML_HiSmall", "IBM_AML_LiSmall", "IBM_AML_HiMedium", "IBM_AML_LiMedium", "AMLSim"]
for x in datasets:
    match x:
        case "Elliptic":
            data_for_optimization = "Elliptic"
            data = elliptic_data
        case "IBM_AML_HiSmall":
            data_for_optimization = "IBM_AML_HiSmall"
            data = IBM_data_HiSmall
        case "IBM_AML_LiSmall":
            data_for_optimization = "IBM_AML_LiSmall"
            data = IBM_data_LiSmall
        case "IBM_AML_HiMedium":
            data_for_optimization = "IBM_AML_HiMedium"
            data = IBM_data_HiMedium
        case "IBM_AML_LiMedium":
            data_for_optimization = "IBM_AML_LiMedium"
            data = IBM_data_LiMedium
        case "AMLSim":
            data_for_optimization = "AMLSim"
            data = AMLSim_data
            
    def save_testing_results_csv(results, path=f"{data_for_optimization}_testing_results.csv"):
        df = pd.DataFrame(results)
        df.to_csv(f"csv_results/{data_for_optimization}_testing_results.csv", index=False)
    model_parameters, testing_results = run_optimization(
        models=['MLP', 'GCN', 'GAT', 'GIN'],
        data=data,
        train_perf_eval=data.train_perf_eval_mask,
        val_perf_eval=data.val_perf_eval_mask,
        test_perf_eval=data.test_perf_eval_mask,
        train_mask=data.train_mask,
        val_mask=data.val_mask,
        data_for_optimization=data_for_optimization
    )
    save_testing_results_csv(testing_results, path=f"{data_for_optimization}_testing_results.csv")

Models:   0%|          | 0/4 [00:00<?, ?model/s][I 2025-11-09 17:04:04,468] A new study created in RDB with name: MLP_optimization on IBM_AML_HiSmall dataset


Study 'MLP_optimization on IBM_AML_HiSmall dataset' found with only 4 trials (< 50). Deleting study.


[I 2025-11-09 17:04:33,033] Trial 0 finished with value: 0.0022128425008236937 and parameters: {'learning_rate': 0.007087092540469284, 'weight_decay': 0.00034121683020441583, 'loss_type': 'focal', 'gamma_focal': 4.004205032396794, 'early_stop_patience': 10, 'early_stop_min_delta': 0.00028858599747638193, 'num_epochs': 242, 'hidden_units': 57}. Best is trial 0 with value: 0.0022128425008236937.
c:\Users\Lambertus\anaconda3\envs\Cuda\lib\site-packages\optuna\trial\_trial.py:678: RuntimeWarning: Inconsistent parameter values for distribution with name "learning_rate"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'step': None, 'low': 0.005, 'high': 0.01, 'log': False}
  warnings.warn(
[W 2025-11-09 17:04:34,192] Trial 1 failed with parameters: {'learning_rate': 0.009747025

ValueError: Loss is NaN, stopping training